# Analysis of Stochastic Linear Programs (SLPs)

In this notebook, we look at some of the properties of Stochastic Linear Programs (SLPs), based on a Julia implementation of a newsvendor problem (energy trading newsvendor problem). 

We first give a compact overview of the concepts for analysis of SLPs, mainly coming for the paper by John Birge (1982). In a second part, we give a basic implementation of the energy trading problem as an SLP. Then in a third part, we illustrate the various concepts based on that problem.

**References**

Birge J. (1982). The value of the stochastic solution in stochastic linear programs with fixed recourse. *Mathematical Programming* **24**: 314-325.


## 1. Analysis of SLPs

### 1.1 Basic program under uncertainty

To eventually discuss the properties of SLPs, we first start by introducing the basic program involving our decisions (here-and-now and recourse) as well as the uncertain parameter. We limit ourselves to the case of fixed recourse. Note that the notations are different from the ones used by John Birge in his paper.

Write $\mathbf{x} \in \mathbb{R}^n$ the here-and-now decision variables, and $\mathbf{y} \in \mathbb{R}^l$ the recourse decision variables. In parallel, $\omega$ is the uncertain parameter. The program we are interested in is

$$
\begin{align}
\min_{\mathbf{x}} \quad & \phi(\mathbf{x}, \omega) \, = \, \mathbf{c}^\top \mathbf{x} \quad + \quad \min_{\textbf{y}} \big[ \, \mathbf{d}^\top \mathbf{y} \, | \,  T(\omega)  \mathbf{x} + W \mathbf{y} = \mathbf{h}(\omega), \, \, \mathbf{y} \geq 0 \, \big] \nonumber\\
\text{s.t} \quad & A \mathbf{x} = \mathbf{b} \nonumber \\
& \mathbf{x} \geq 0 \nonumber
\end{align}
$$

### 1.2 Definitions of important quantities

If you had perfect information on $\omega$, and could know what its realization may be, then you would plug that value of the realization of $\omega$ in the above problem and solve it. Extending that idea, since being provided with a number of scenarios $\omega_k$ and with associated probabilities $p_k$, one could solve the problem for each scenario $\omega_k$, as if you knew this is what will happen, and then obtain an expectation based on the $p_k$ probabilities. This is what is called the *wait-and-see* solution (WS), with

$$\text{WS} = \mathbb{E}_\omega \left[ \min_{\mathbf{x}} \, \phi(\mathbf{x}, \omega) \right] $$

which in practice, based on the scenarios available would become

$$\text{WS} = \sum_k p_k \left[ \min_{\mathbf{x}} \,  \phi(\mathbf{x}, \omega_k) \right] $$

As an alternative, even if $\omega$ is not known in advance, while a number of scenarios $\omega_k$ are available, one could also think of solving the original problem with a single value for $\omega$, this single value being its expectation,

$$\mathbb{E}[\omega] = \sum_k p_k \omega_k $$

This eventually yields the so-called *expected value* (EV) problem:

$$\text{EV} = \min_{\mathbf{x}} \, \phi \left(\mathbf{x}, \mathbb{E}[\omega] \right) $$

We usually write $\mathbf{x}(\bar{\omega})$ that solution, using $\bar{\omega}$ as a notation for the expectation of $\omega$.

If implementing that approach, we may then look at what would be the cost of having made such here-and-now decision while any of the scenarios $\omega_k$ could actually realize. The expectation of this cost is referred to as the *expected result of the expected value* problem (EEv, in short), and is defined by

$$\text{EEV} = \mathbb{E}_\omega \left[ \phi \left(\mathbf{x}(\bar{\omega}), \omega \right) \right] $$

Again, based on the available scenarios, this gives

$$\text{EEV} = \sum_k p_k \phi \left(\mathbf{x}(\bar{\omega}), \omega_k \right) $$

Obviously, we have now seen that it may be more appropriate to solve the stochastic linear program with fixed recourse to find a solution to the above program, in expectation. This is also referred to as the *recourse problem* (RP, as for the case of Birge's paper), with

$$\text{RP} = \min_{\mathbf{x}} \, \mathbb{E}_\omega \left[ \phi(\mathbf{x}, \omega) \right] $$

which in practice, based on the scenarios available would become

$$\text{RP} = \min_{\mathbf{x}} \,  \sum_k p_k \phi(\mathbf{x}, \omega_k)  $$

Finally, we usually define the difference between the wait-and-see solution and the stochastic linear program, as the *expected value of perfect information* (EVPI), i.e.

$$ \text{EVPI} = \text{RP}- \text{WS} $$


### 1.3 Relationship between these quantities

Maybe the most important base result to analyse stochastic linear program is the following:

$$ \text{EEV} \geq \text{RP} \geq \text{WS} \geq \text{EV} $$

This means that the objective function value obtained when solving the expected-value problem is at most that for the wait-and see problem, which is at most that for the stochastic linear program (recourse problem), which is at most the expected result of the expected value. This **does not** provide a ranking of the solutions, since please remember that the objective function values may not be directly comparable. For instance, the objective function of the expected-value problem completely overlooks what the recourse costs may be.

Now, a first important result that can be deduced, is that the expected value of perfect information (EVPI) is at worst 0, and at best the difference between the EEV and the EV, i.e.,

$$ 0 \leq \text{EVPI} \leq \text{EEV} - \text{EV} $$

However, we would prefer to have something that links RP and EEV, since we are generally interested on how well we may solve the problem with using a stochastic linear program instead of the expected value solution. For that, we define the *value of the stochastic solution* (VSS), as

$$ \text{VSS} = \text{EEV} - \text{RP} $$

And, we can deduce that

$$ 0 \leq \text{VSS} \leq \text{EEV} - \text{EV} $$

This is a key result of stochastic linear programming, since it tells that **the stochastic linear program will always be at least as good at the expected-value solution!**


## 2. Newsvendor problem as a SLP (energy trading newsvendor problem)

When selling renewable energy generation in electricity markets: one should place a bid in advance (the day before). Necessarily, one cannot know how much a wind farm or solar power plant will produce the day after, with full certainty. However, if one produces more of less energy than in the contract, there is a penalty. One therefore needs to find the optimal energy quantity to offer, in view of that uncertainty, as well as the penalties for producing more or less than the contract.

In the following, we first go through the way the problem is set up and then reformulated as an SLP. Then, we show how to solve that problem in Julia. 

### 2.1 Problem setup and formulation

When starting with a newsvendor problem, we first need to identify the here-and-now decisions, and the recourse decisions. 

In that version of the problem, a renewable energy trader (for instance) has to decide in advance (one day for the next) on a quantity $x$ of energy to sell on the electricity market. It is not the demand here that is uncertain, but how much the renewable energy asset will produce eventually. The trader has a number $K$ of scenarios for that production, which we denote by $\omega_k$, $k=1,\ldots,K$, and with probability $p_k$ ($\sum_k p_k = 1$). In the examples below, we consider that these scenarios come from a Gamma distribution, with parameters $k=30$, and $\theta=1.5$. The expected energy generation therefore is $\mu = k\theta = 45$ MWh

The here-and-now decision is that quantity $x$. The unit price received when selling the quantity $x$ on the electricity market is $c$ (ex: $c=$ 100 DKK/MWh).

Now, whenever the actual energy production is less than the contracted quantity $x$, the trader has to compensate by buying the quantity missing from another market at a price $c+\delta$, e.g. $\delta=$ 20 DKK/MWh. And, if producing more energy than the contracted quantity $x$, the market will take it, but at a price $c-\eta$ that is lower that original price $c$ on the electricity market (e.g., $\eta=$ 10 DKK/MWh). All in all, that means that, compared the case where the trader could perfectly predict how much will be produced, there are then 2 potential cases:
- if energy production is more than the contracted value $x$, there will be an opportunity cost of $\eta$ per unit of energy above $x$, 
- if energy production is less than the contracted value $x$, there will be a direct loss of $\delta$ for any unit of energy below $x$.

Based on these two cases, one could define the asymmetric loss function that was discussed when solving the newsvendor problem analytically. Looking at the problem as an SLP, these cases relate to a number of recourse decisions, i.e., the quantity of energy contracted/produced $y_1$, the quantity of energy missing $y_2$ (if production is less than the contract $x$), and the surplus of energy $y_3$ (in case the production is less than the contract $x$). Clearly, if $y_2=0$, then $y_3\geq0$, and if $y_3=0$, then $y_2\geq0$.

Normally, a trader would want to maximize revenue. However, for convenience, we want to write it as a minimization problem (hence, minus the revenue). Let us then formulate this problem as an SLP, as

$$
\begin{align}
\min_{x} && (-c) x + \mathbb{E}_\omega \left[ Q(x, \omega) \right] \nonumber\\
&& Q(x,\omega) & = \min_{y_1,y_2,y_3} & (c+\delta) y_2 - (c-\eta) y_3 \nonumber \\
&&             & \qquad \text{s.t.}          & y_1 + y_2 = x \nonumber \\
&&             &                      & y_1 + y_3 = \omega \nonumber \\
&&             &                      & x,y_1,y_2,y_3 \geq 0 \nonumber
\end{align}
$$

Let us detail why the formulation, using 3 recourse variables $y_1$, $y_2$ and $y_3$ is very convenient. We have to consider 2 case:
- if there is an energy generation surplus, this means we do not have missing energy, so $y_2=0$. The 2 constraints then are
$$
\begin{align}
&y_1 = x \nonumber \\
&y_1 + y_3 = \omega \nonumber \\
& x,y_1,y_3 \geq 0 \nonumber
\end{align}
$$
Therefore, the variable $y_1$ gives the energy sold (contract) through the market, and $y_3$ defines the surplus (i.e., the difference between $x$ and $\omega$).
- if there is an energy generation shortage, this means we do not have a surplus, so $y_3=0$. The 2 constraints then are
$$
\begin{align}
&y_1 + y_2 = x \nonumber \\
&y_1 = \omega \nonumber \\
& x,y_1,y_2 \geq 0 \nonumber
\end{align}
$$
Therefore, the variable $y_1$ gives the energy actually produced ($\omega$), and $y_2$ defines the missing energy (i.e., the difference between $y_1$ and $x$).

In practice, since we have a number of scenarios and associated probabilities, the SLP problem in its deterministic form becomes

$$
\begin{align}
\min_{x,y_{1,k},y_{2,k},y_{3,k}} \quad     & (-c) x + \sum_{k=1}^K p_k \left( (c+\delta) y_{2,k} - (c-\eta) y_{3,k} \right) \nonumber \\
\text{s.t.} \quad  & y_{1,k} + y_{2,k} = x, \quad k=1,\ldots,K \nonumber \\
                   & y_{1,k} + y_{3,k} = \omega_k,  \quad k=1,\ldots,K \nonumber \\
                   & x,y_{1,k},y_{2,k},y_{3,k} \geq 0 \nonumber
\end{align}
$$

Since the here-and-now decision is scenario-independent (non-anticipativity constraint), we only have one decision variable $x$ in the above problem. However, since the recourse variable $y_1$, $y_2$ and $y_3$ are scenario dependent, we have to find an optimal value for each and every scenario $\omega_k$. This is why an additional subscript $k$ appears in notations for those decision variables.

Finally, if aiming to write that problem in a format similar to what we obtained in Section 1, the here-and-now constraints can be removed (i.e., related to $A$ and $\mathbf{b}$) as there are none. In parallel, one needs to write the recourse cost vector as

$$ \mathbf{d} = [0 \, \,  (c+\delta) \, \, -(c-\eta)]^\top $$

the matrix $T$ and $W$ for the recourse constraints as

$$ T = \left( \begin{array}{c}
0\\
-1
\end{array}
\right)
$$

and 

$$ W = \left( \begin{array}{ccc}
1 & 1 & 0\\
1 & 0 & 1\\
\end{array}
\right)
$$

as well as the right-hand side $h$ as

$$ h_k = \left( \begin{array}{c}
0\\
\omega_k
\end{array}
\right)
$$

In the above, both $T$ and $W$ matrices are independent of $\omega_k$, hence we have a stochastic linear program with fixed recourse.





### 2.2 Solving in Julia with JuMP and GPLK

We consider here an example application, based on the cost values and description of the uncertain parameter $\omega$ in the above. This means that
- energy is sold at $c=$ 100 DKK/MWh
- the underage penalty is $\eta=$ 10 DKK/MWh
- the overage penalty is $\delta=$ 20 DKK/MWh
- the uncertain parameter follows a Gamma distribution, $\omega \sim \text{Gamma}(30,1.5)$

For the implementation, we fix the seed of the random number generator (to be sure we get the same results as in the lecture, and when re-running the notebook). And, we use 100 equiprobable scenarios for this example. This number will be varied later on, when focusing on the effect of sampling.

The code and results are gathered in the following:

In [1]:
#Import necessary Julia packages
using LinearAlgebra, JuMP, GLPK, Distributions, Random

#Declare model and optimizer
nvmodel = Model()
set_optimizer(nvmodel, GLPK.Optimizer)

#Define parameters
c = 100
δ = 20
η = 10
ns = 5 # number of scenarios

#Obtain scenarios (they are all equiprobable, with probability 1/ns)
d = Gamma(30,1.5)
Random.seed!(114) # fix the seed of the random number generator, to be sure we get the same
ω = rand(d,ns) 

#Define variables
@variable(nvmodel, x)
@variable(nvmodel, y_1[1:ns])
@variable(nvmodel, y_2[1:ns])
@variable(nvmodel, y_3[1:ns])

#Define Constraints
@constraint(nvmodel, [i=1:ns], - x + y_1[i] + y_2[i] == 0)
@constraint(nvmodel, [i=1:ns], y_1[i] + y_3[i] == ω[i])
@constraint(nvmodel, [i=1:ns], y_1[i] >= 0)
@constraint(nvmodel, [i=1:ns], y_2[i] >= 0)
@constraint(nvmodel, [i=1:ns], y_3[i] >= 0)
@constraint(nvmodel, x >= 0)

#Define Objective
@objective(nvmodel, Min, (-c) * x + (1/ns) * sum( ((c+δ)*y_2[i] - (c-η)*y_3[i]) for i in 1:ns))

#Run the opimization
optimize!(nvmodel)

The scenarios we have as input (to described potential energy generation) are:

In [2]:
println(ω)

[59.41941568329372, 43.616974047760934, 50.21235165865798, 50.463473672810096, 36.519643914844266]


The here-and-now decision $x$ is 

In [3]:
value.(x)

43.616974047760934

We can also look at all the recourse decisions, for each and every scenario:

In [4]:
value.(y_1)

5-element Vector{Float64}:
 43.616974047760934
 43.616974047760934
 43.616974047760934
 43.616974047760934
 36.519643914844266

In [5]:
value.(y_2)

5-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 7.097330132916667

In [6]:
value.(y_3)

5-element Vector{Float64}:
 15.802441635532787
  0.0
  6.595377610897046
  6.846499625049162
  0.0

Finally, at optimality, the objective function value is:

In [7]:
objective_value(nvmodel)

-4717.759221272714

Remember that one should cautious when looking at this objective function value, as it mixes costs related to the here and now decision (here, -43.62 $\times$ 100=-4262 DKK) with the expected recourse costs.

## 3. Analysing properties of an SLP based on the newsvendor example

Let us follow the line of thoughts of Section 1, in term of defining and solving the various problems. 

We start with the wait-and-see problem, which consists in solving the original problem for each of the scenarios individually, and then taking the expectation. Here, since our 5 scenarios are equiprobable, that will simply be the average.

In [8]:
#Import necessary Julia packages
using LinearAlgebra, JuMP, GLPK, Distributions, Random

#Define parameters
c = 100
δ = 20
η = 10
ns = 5 # number of scenarios

#Obtain scenarios (they are all equiprobable, with probability 1/ns)
d = Gamma(30,1.5)
Random.seed!(114) # fix the seed of the random number generator, to be sure we get the same
ω = rand(d,ns)

# Declare vectors to store (here-and-now) decisions and objective function values
xv = zeros(ns)
ov = zeros(ns)

for i =1:ns # loop over scenarios
    #Declare model and optimizer
    wsmodel = Model()
    set_optimizer(wsmodel, GLPK.Optimizer)

    #Define variables
    @variable(wsmodel, x)
    @variable(wsmodel, y_1)
    @variable(wsmodel, y_2)
    @variable(wsmodel, y_3)

    #Define Constraints
    @constraint(wsmodel, - x + y_1 + y_2 == 0)
    @constraint(wsmodel, y_1 + y_3 == ω[i])
    @constraint(wsmodel, y_1 >= 0)
    @constraint(wsmodel, y_2 >= 0)
    @constraint(wsmodel, y_3 >= 0)
    @constraint(wsmodel, x >= 0)

    #Define Objective
    @objective(wsmodel, Min, (-c) * x + (c+δ) * y_2 - (c-η) * y_3 )
    #Run the opimization
    optimize!(wsmodel)
    
    #Extract solutions
    xv[i] = value.(x)
    ov[i] = objective_value(wsmodel)
end

println("WS here-and-now decision: ", mean(xv))
println("WS = ", mean(ov))


WS here-and-now decision: 48.0463717954734
WS = -4804.637179547341


The solution is actually straightforward, and the use of that code could be avoided. Indeed, if one knew the energy generation that would happen, the best decision would be to use this as here-and-now decision. Hence, in each scenario, the best is to use $\omega_k$ as the here-and-now decision. The objective value we obtain is WS = -4804.64. We can already see that WS $\leq$ RP. 

And, the expected value of perfect information (EVPI) is

$$\text{EVPI} = -4717.76 - (-4804.64) = 86.88$$

This means that, in expectation for that problem, you would earn 86.88 DKK if you could know in advance which value of $\omega$ was to realize.

Let us now look at the expected-value (EV) solution. We can use a similar approach for the code, expect that we first take the average of the scenarios and then solve a single program.

In [9]:
#Import necessary Julia packages
using LinearAlgebra, JuMP, GLPK, Distributions, Random

#Define parameters
c = 100
δ = 20
η = 10
ns = 5 # number of scenarios

#Obtain scenarios (they are all equiprobable, with probability 1/ns)
d = Gamma(30,1.5)
Random.seed!(114) # fix the seed of the random number generator, to be sure we get the same
ω = rand(d,ns)
ωm = mean(ω)

#Declare model and optimizer
evmodel = Model()
set_optimizer(evmodel, GLPK.Optimizer)

#Define variables
@variable(evmodel, x)
@variable(evmodel, y_1)
@variable(evmodel, y_2)
@variable(evmodel, y_3)

#Define Constraints
@constraint(evmodel, - x + y_1 + y_2 == 0)
@constraint(evmodel, y_1 + y_3 == ωm)
@constraint(evmodel, y_1 >= 0)
@constraint(evmodel, y_2 >= 0)
@constraint(evmodel, y_3 >= 0)
@constraint(evmodel, x >= 0)

#Define Objective
@objective(evmodel, Min, (-c) * x + (c+δ) * y_2 - (c-η) * y_3 )
#Run the opimization
optimize!(evmodel)

println("WS here-and-now decision: ", value.(x))
println("WS = ", objective_value(evmodel))

WS here-and-now decision: 48.0463717954734
WS = -4804.63717954734


Here again, the solution is obvious, since we solve the program as if we considered we knew the expected value for $\omega$ was what is going to happen. Hence, we have
$$ x_{\text{EV}} = \mathbb{E}[\omega] = \sum_k p_k \omega_k = 48.05 $$

The objective value we obtain is EV = -4804.64. This verifies the general case such that EV $\leq$ WS, since we actually have EV $=$ WS.

We will now calculate the expected result of the expected-value solution (EEV):

In [10]:
#Import necessary Julia packages
using LinearAlgebra, JuMP, GLPK, Distributions, Random

#Define parameters
c = 100
δ = 20
η = 10
ns = 5 # number of scenarios

#Obtain scenarios (they are all equiprobable, with probability 1/ns)
d = Gamma(30,1.5)
Random.seed!(114) # fix the seed of the random number generator, to be sure we get the same
ω = rand(d,ns)
ωm = mean(ω)

# Declare the EV here-and-now decision, as well as vector of obvective values over scenarios
x = ωm
ov = zeros(ns)

for i =1:ns # loop over scenarios
    if (ω[i]>x)
        y_1 = x
        y_2 = 0
        y_3 = ω[i]-x
        ov[i] = (-c) * x + (c+δ) * y_2 - (c-η) * y_3
    else
        y_1 = ω[i]
        y_2 = x-ω[i]
        y_3 = 0
        ov[i] = (-c) * x + (c+δ) * y_2 - (c-η) * y_3
    end    
end

println("EEV = ", mean(ov))


EEV = -4708.90042577729


There, specifically, we can see that EEV $\geq$ RP. Overall, we retrieve the relationship described in Section 1.3, i.e. such that

$$\text{EEV} \geq \text{RP} \geq \text{WS} \geq \text{EV}$$

Finally, we find that the value of the stochastic solution is 

$$\text{VSS} = \text{EEV}-\text{RP} = -4708.90 - (-4717.76) = 8.86 $$

As expected, it is such that VSS $\geq0$.